# Data for project 2:

In [66]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import os
from collections import Counter
import matplotlib.pyplot as plt

In [29]:
os.chdir('/Users/carlahugod/Desktop/UNI/3sem/ML/Project1')

In [91]:
filename = 'smoking_driking_dataset_Ver01.csv'
data = pd.read_csv(filename)

X = data.values

attriubutes = np.asarray(data.columns)
print(attriubutes)

cols = range(0,24) # 24 columns in the data set
X = X[:,cols] # defining the data as a matrix

# Replacing strings with numbers. Starting with gender:
X[X == 'Female'] = 0
X[X == 'Male'] = 1

# Replacing Y = yes with 1 and N = no with 0 in regards to drinking
X[X == 'Y'] = 1
X[X == 'N'] = 0



['sex' 'age' 'height' 'weight' 'waistline' 'sight_left' 'sight_right'
 'hear_left' 'hear_right' 'SBP' 'DBP' 'BLDS' 'tot_chole' 'HDL_chole'
 'LDL_chole' 'triglyceride' 'hemoglobin' 'urine_protein'
 'serum_creatinine' 'SGOT_AST' 'SGOT_ALT' 'gamma_GTP' 'SMK_stat_type_cd'
 'DRK_YN']


In [92]:
# Remove outliers: waistline, eye sight

# eye sight, 9.9 means blind, therefore replacing with 0. That is columnb 5 and 6 starting from 0
X[X[:,5] == 9.9] = 0
X[X[:,6] == 9.9] = 0

# Create a for loop to find all the indices where waistline is 999
indices = []
c = 0
for i in range(0, len(X[:,4])):
    if X[i,4] == 999:
        indices.append(i)
    if X[i,4] < 1:
        c += 1
        indices.append(i)

# Now removing the rows with the indices found above
X = np.delete(X, indices, axis=0)

# Doing the samr for SGOT_AST
indices = []
c = 0
for i in range(0, len(X[:,19])):
    if X[i,19] > 3000:
        c += 1
        indices.append(i)

X = np.delete(X, indices, axis=0)

In [93]:
# Standardize the data, execept last two columns
for i in [0,1,2,3,4,5,6,9,10,11,12,13,14,15,16,17, 18,19,20,21,23]:
    X[:,i] = (X[:,i] - X[:,i].mean()) / X[:,i].std()
# Here everything is standardized, except the last two columns, 

# Now I will only acces the last column and standardize it:
X[:,23] = (X[:,22] - X[:,22].mean()) / X[:,22].std()

In [95]:
one_out_of_K_cols_dict = {'encoding1':'hear_left', 'encoding2':'hear_right', 'encoding4':'SMK_stat_type_cd'}

# getting indices of collums that needs encoding
indices_list = [index for index, element in enumerate(attriubutes) if element in one_out_of_K_cols_dict.values()]

key_iterator = iter(one_out_of_K_cols_dict)

# For loop for creating encoding matrix for all keys in one_out_of_K_cols_dict
for i in range(len(indices_list)):
    K = int(X[:,indices_list[i]].max())+1
    encoding = np.zeros((X[:,indices_list[i]].size, K))
    encoding[np.arange(X[:,indices_list[i]].size),X[:,indices_list[i]].astype(int)] = 1
    # deleting first column, because values incoded goes from 1:n and not 0:n, so K=K+1, for this to work.
    encoding = encoding[:,1:]
    # Storing encoding as value in dict
    one_out_of_K_cols_dict[str(next(key_iterator))] = encoding

# Deleting old data columns
for j in range(len(indices_list)):
    data = np.delete(X, indices_list[j]-j, axis=1)

# Remember that last key (encoding for 'SMK_stat_type_cd') will be the first colums in data matrix
# smoking staus first, hearing right, hearing left. Smoking status is the first 3 columns. SMT1 is the first column.

for key in one_out_of_K_cols_dict:
    for k in range(one_out_of_K_cols_dict[str(key)].shape[1]):
        X = np.insert(X, k, one_out_of_K_cols_dict[str(key)][:,k], axis=1)